In [1]:
import numpy as np
import open3d as o3d
import torch

from superprimitive_fusion.scanner import capture_spherical_scans
from superprimitive_fusion.utils import bake_uv_to_vertex_colours
from superprimitive_fusion.mesh_fusion_utils import get_mesh_components, show_mesh_boundaries
from superprimitive_fusion.mesh_fusion import fuse_meshes

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [30]:
mustard = o3d.io.read_triangle_mesh("../data/posed-meshes/mustard-bottle/mustard-bottle.ply", enable_post_processing=True)

table = o3d.io.read_triangle_mesh("../data/posed-meshes/table/table.obj", enable_post_processing=True)

drill = o3d.io.read_triangle_mesh("../data/posed-meshes/power-drill/power-drill.obj", enable_post_processing=True)

bleach = o3d.io.read_triangle_mesh("../data/posed-meshes/bleach/bleach.obj", enable_post_processing=True)

pitcher = o3d.io.read_triangle_mesh("../data/posed-meshes/pitcher/pitcher.obj", enable_post_processing=True)

mug = o3d.io.read_triangle_mesh("../data/posed-meshes/mug/mug.obj", enable_post_processing=True)

clamp = o3d.io.read_triangle_mesh("../data/posed-meshes/extra-large-clamp/extra-large-clamp-leaning.obj", enable_post_processing=True)

# bake_uv_to_vertex_colours(mesh)

# mesh.compute_vertex_normals()

# bb = mesh.get_minimal_oriented_bounding_box()
# scale = np.mean(bb.get_max_bound())

In [38]:
meshes = [mustard, table, drill, bleach, pitcher, mug, clamp]
o3d.visualization.draw_geometries(meshes)

In [41]:
# {
# 	"class_name" : "ViewTrajectory",
# 	"interval" : 29,
# 	"is_loop" : false,
# 	"trajectory" : 
# 	[
# 		{
# 			"boundingbox_max" : [ 0.89999997615814209, 0.44999998807907104, 0.97050297260284424 ],
# 			"boundingbox_min" : [ -0.89999997615814209, -0.44999998807907104, 0.0 ],
# 			"field_of_view" : 60.0,
# 			"front" : [ 0.033181979928996505, 0.89223756678007571, 0.45034551250607779 ],
# 			"lookat" : [ 0.0088310242409319843, 0.0013222086626097952, 0.81265236644826389 ],
# 			"up" : [ -0.0093729695001894316, -0.45029602336870767, 0.89283012873730749 ],
# 			"zoom" : 0.11999999999999962
# 		}
# 	],
# 	"version_major" : 1,
# 	"version_minor" : 0
# }

In [39]:
import numpy as np
import open3d as o3d

def render_rgb_view(
    mesh: o3d.geometry.TriangleMesh,
    cam_centre=(0.3, 0.3, 0.0),
    look_dir=(0.0, 0.0, 0.0),
    *,
    width_px: int = 360,
    height_px: int = 240,
    fov: float = 70.0,
    bg_rgba=(1.0, 1.0, 1.0, 1.0),  # white background
) -> np.ndarray:
    """Render an RGB image of `mesh` from a given camera pose."""
    renderer = o3d.visualization.rendering.OffscreenRenderer(width_px, height_px)

    # Simple material: unlit if vertex colors exist, otherwise lit shading
    mtl = o3d.visualization.rendering.MaterialRecord()
    mtl.shader = "defaultUnlit" if mesh.has_vertex_colors() else "defaultLit"
    renderer.scene.add_geometry("mesh", mesh, mtl)
    renderer.scene.set_background(bg_rgba)

    # Camera: vertical FOV in degrees, look-at with Z-up
    aspect = width_px / height_px
    cam = renderer.scene.camera
    cam.set_projection(
        fov, aspect, 0.01, 1000.0,
        o3d.visualization.rendering.Camera.FovType.Vertical
    )
    cam.look_at(look_dir, cam_centre, [0, 0, 1])

    # Render to image and return RGB
    img = renderer.render_to_image()
    rgb = np.asarray(img)
    if rgb.ndim == 3 and rgb.shape[2] == 4:  # drop alpha if present
        rgb = rgb[:, :, :3]
    return rgb
